In [1]:
import pandas as pd
df = pd.read_csv("C:/Users/joeco/Python/fb-ref-data/data/181220_2021_aggregated_player_data.csv")

In [2]:
list_of_pl_dms = ["Declan Rice","Oliver Norwood","Ruben Neves","Granit Xhaka","Isaac Hayden","James McCarthy",
                 "Scott McTominay","Rodri","Josh Brownhill","Andre-Frank Zambo Anguissa","Fred",
                 "Wilfred Ndidi","Conor Gallagher","Kalvin Phillips","Allan","Douglas Luiz","Yves Bissouma",
                 "Pierre Højbjerg","N'Golo Kanté","Fabinho","Thomas Partey","Abdoulaye Doucouré","Oriol Romeu",
                 "Tomáš Souček"]
narrow_list = ["Declan Rice","Isaac Hayden","Jordan Henderson","Rodri","Douglaz Luiz","Wilfred Ndidi","Yves Bissouma",
                 "Kalvin Phillips","N'Golo Kanté","Fred","Pierre Højbjerg","Allan"]
tenth_feb_21_list = ["Declan Rice","Scott McTominay","Tomáš Souček"]

In [3]:
base_stats = ["yellow_cards","red_cards","pressures","tackles","interceptions","blocks","sca","passes_completed",
             "passes_attempted","passes_prog_dist","carries","carries_prog_dist","dribbles_completed","dribbles_attempted",
              "passes_total_dist","passes_completed_short","passes_attempted_short","passes_completed_medium",
              "passes_attempted_medium","passes_completed_long","passes_attempted_long","assisted_shots",
              "passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","prog_passes",
             "through_balls","passes_under_pressure","passes_left_foot","passes_right_foot","tackles_won","tackles_def_3rd",
              "tackles_att_3rd","dribblers_tackles","dribblers_attempted_vs","dribbled_past","pressure_regains",
             "blocked_shots","blocked_shots_on_target","blocked_passes","tackles_interceptions","clearances","errors",
             "players_dribbled_past","carry_distance","pass_target_count","passes_received","miscontrols","dispossessed",
              "fouls","times_fouled","ball_recoveries","goals","assists"]

In [22]:
def percentage_of(row,col1,col2,one_minus=False):
    a = int(row[col1])
    b = int(row[col2])
    if b == 0:
        return 0
    c = round((a/b)*100,1)
    if one_minus:
        return 100-c
    else:
        return c

def create_per_90_stat(row,stat_col):
    total_mins = int(row["total_mins"])
    stat = round(float(row[stat_col]),2)
    total_90s = round(float(total_mins/90),2)
    per_90s_stat = round(float(stat/total_90s),2)
    return per_90s_stat

def compare_dm_stats(df,players,stat_list,per90=False):
    base_info = ["team","name","total_mins"]
    df_players = df.query("name in @players")
    df_narrowed = df_players[base_info + stat_list].copy()
    main_def_stats = ["tackles","interceptions","blocks","tackles_won","blocked_shots","blocked_passes",
                     "tackles_interceptions","clearances","errors","fouls","ball_recoveries"]
    main_poss_stats = ["passes_prog_dist","passes_into_final_third","passes_into_penalty_area","passes_under_pressure",
                      "times_fouled","carry_distance"]
    main_att_stats = ["sca","assisted_shots","through_balls","goals","assists"]
    main_misc_stats = ["yellow_cards","miscontrols","dispossessed"]
    main_stats = main_def_stats + main_poss_stats + main_att_stats + main_misc_stats
    
    df_copy = df_narrowed.copy()
    df_out = df_copy[base_info + main_stats].copy()
    if per90:
        for stat in main_stats:
            df_out[str(stat) + "_p90"] = df_copy.apply(create_per_90_stat,args=(stat,),axis=1)
            df_out.drop(stat,inplace=True,axis=1)
            
    
    df_out["success_pressure_%"] = df_copy.apply(percentage_of,args=("pressure_regains","pressures"),axis=1)
    df_out["pass_success_%"] = df_copy.apply(percentage_of,args=("passes_completed","passes_attempted"),axis=1)
    df_out["dribble_success_%"] = df_copy.apply(percentage_of,args=("dribbles_completed","dribbles_attempted"),axis=1)
    df_out["prog_pass_%"] = df_copy.apply(percentage_of,args=("prog_passes","passes_attempted"),axis=1)
    
    df_out["dribblers_stopped_%"] = df_copy.apply(percentage_of,args=("dribbled_past","dribblers_attempted_vs",True),axis=1)
    
    return df_out

def sum_ranks(row,cols):
    total = 0
    for rank in cols:
        total += int(row[rank])
    return total

def convert_to_ranks(df,per90=False):
    base_list = ["team","name","total_mins"]
    if per90 == True:
        descending_list = ["errors_p90","fouls_p90","yellow_cards_p90","miscontrols_p90","dispossessed_p90"]
    else:
        descending_list = ["errors","fouls","yellow_cards","miscontrols","dispossessed"]
    df_output = df[base_list].copy()
    stat_list = [x for x in df.columns.values if x not in base_list]
    for stat in stat_list:
        if stat in descending_list:
            df_output[stat] = df[stat].rank(method='min')
        else:
            df_output[stat] = df[stat].rank(method='min',ascending=False)
        
    df_output["sum_ranks"] = df_output.apply(sum_ranks,args=(stat_list,),axis=1)
    df_output.sort_values(by="sum_ranks",ascending=True,axis=0,inplace=True)
    return df_output
        
    

In [20]:
df_out = compare_dm_stats(df,tenth_feb_21_list,base_stats)

In [21]:
df_out.head()

,team,name,total_mins,tackles,interceptions,blocks,tackles_won,blocked_shots,blocked_passes,tackles_interceptions,...,goals,assists,yellow_cards,miscontrols,dispossessed,success_pressure_%,pass_success_%,dribble_success_%,prog_pass_%,dribblers_stopped_%
57,Manchester United,Scott McTominay,1234,28,20,37,17,9,28,48,...,4,1,0,11,17,32.3,86.6,76.5,7.8,28.0
103,West Ham United,Declan Rice,2070,57,44,41,43,6,35,101,...,0,1,2,18,16,31.5,85.3,86.2,6.9,48.8
309,West Ham United,Tomáš Souček,2069,47,25,30,28,7,23,72,...,8,0,3,17,11,27.0,74.5,66.7,7.4,25.5


In [23]:
df_ranks = convert_to_ranks(df_out)

In [24]:
df_ranks

,team,name,total_mins,tackles,interceptions,blocks,tackles_won,blocked_shots,blocked_passes,tackles_interceptions,...,assists,yellow_cards,miscontrols,dispossessed,success_pressure_%,pass_success_%,dribble_success_%,prog_pass_%,dribblers_stopped_%,sum_ranks
103,West Ham United,Declan Rice,2070,1.0,1.0,1.0,1.0,3.0,1.0,1.0,...,1.0,2.0,3.0,2.0,2.0,2.0,1.0,3.0,1.0,49
57,Manchester United,Scott McTominay,1234,3.0,3.0,2.0,3.0,1.0,2.0,3.0,...,1.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,62
309,West Ham United,Tomáš Souček,2069,2.0,2.0,3.0,2.0,2.0,3.0,2.0,...,3.0,3.0,2.0,1.0,3.0,3.0,3.0,2.0,3.0,66


In [25]:
df_ranks.to_csv("C:/Users/joeco/Python/fb-ref-data/data/100221_2021_rice_mctominay_soucek.csv",index=False)

In [26]:
# Per 90
df_out = compare_dm_stats(df,tenth_feb_21_list,base_stats,per90=True)

In [27]:
df_out.head()

,team,name,total_mins,tackles_p90,interceptions_p90,blocks_p90,tackles_won_p90,blocked_shots_p90,blocked_passes_p90,tackles_interceptions_p90,...,goals_p90,assists_p90,yellow_cards_p90,miscontrols_p90,dispossessed_p90,success_pressure_%,pass_success_%,dribble_success_%,prog_pass_%,dribblers_stopped_%
57,Manchester United,Scott McTominay,1234,2.04,1.46,2.70,1.24,0.66,2.04,3.50,...,0.29,0.07,0.00,0.80,1.24,32.3,86.6,76.5,7.8,28.0
103,West Ham United,Declan Rice,2070,2.48,1.91,1.78,1.87,0.26,1.52,4.39,...,0.00,0.04,0.09,0.78,0.70,31.5,85.3,86.2,6.9,48.8
309,West Ham United,Tomáš Souček,2069,2.04,1.09,1.30,1.22,0.30,1.00,3.13,...,0.35,0.00,0.13,0.74,0.48,27.0,74.5,66.7,7.4,25.5


In [28]:
df_ranks = convert_to_ranks(df_out,per90=True)
df_ranks

,team,name,total_mins,tackles_p90,interceptions_p90,blocks_p90,tackles_won_p90,blocked_shots_p90,blocked_passes_p90,tackles_interceptions_p90,...,assists_p90,yellow_cards_p90,miscontrols_p90,dispossessed_p90,success_pressure_%,pass_success_%,dribble_success_%,prog_pass_%,dribblers_stopped_%,sum_ranks
57,Manchester United,Scott McTominay,1234,2.0,2.0,1.0,2.0,1.0,1.0,2.0,...,1.0,1.0,3.0,3.0,1.0,1.0,2.0,1.0,2.0,50
103,West Ham United,Declan Rice,2070,1.0,1.0,2.0,1.0,3.0,2.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,1.0,55
309,West Ham United,Tomáš Souček,2069,2.0,3.0,3.0,3.0,2.0,3.0,3.0,...,3.0,3.0,1.0,1.0,3.0,3.0,3.0,2.0,3.0,72


In [29]:
df_out.to_csv("C:/Users/joeco/Python/fb-ref-data/data/100221_2021_rice_mctominay_soucek_p90.csv",index=False)